# Top Trendings do Twitter
**Objetivos:** Desenvolver uma aplicação para monitorar o top trending e seu devido volume na rede social com o passar do tempo.

## Funcionalidades
**Requisitos funcionais obrigatorios**:
- [x] Utilizar a api do twitter para consumir o top trending e seu devido volume.
- [ ] Armazenar o histórico das trendings em um banco de dados noSQL.
- [ ] Utilizar alguma biblioteca gráfica para exibir os dados.
**Requisito funcional opcional**:
- [ ] Utilizar IA para análise de sentimento dos tweets referentes aos top trendings

### Instação das Bibliotecas
Para instalar as bibliotecas, será usado o comando abaixo, do qual lê um arquivo de texto que se encontra na raiz do projeto e possui as bibliotecas usadas neste projeto.

In [ ]:
! python -m pip install -r requirements.txt

### Imports

In [ ]:
import json

import requests

from modules import mongoDb
from modules import fileToDic as fd
from modules import apiKeys as api

In [ ]:
file = open("input/WOEID.txt")

In [ ]:
countries = {}

In [ ]:
search_url = "https://api.twitter.com/1.1/trends/place.json"

In [ ]:
bearer_token = api.key['bearer_token']

In [ ]:
countries = fd.dictionary(file, countries)

In [ ]:
query_params = {'id': countries['BRAZIL']}

In [ ]:
def bearer_oauth(r):
  '''
  Autenticação usando o bearer token
  '''
  r.headers["Authorization"] = f"Bearer {bearer_token}"
  r.headers["User-Agent"] = "v2RecentSearchPython"

  return r

In [ ]:
def connect_to_endpoint(url, params):
  '''
  Se conecta a url dada com o parametros dado.
  Consultar a documentação para saber o parametros nessario na pesquisa
  '''
  response = requests.get(url, auth=bearer_oauth, params=params)
  print(response.status_code)
  if response.status_code != 200:
    raise Exception(response.status_code, response.text)
        
  return response.json()

In [ ]:
json_response = connect_to_endpoint(search_url, query_params)
print(json.dumps(json_response, indent=4, sort_keys=True))